### [인덱스 다루기 - MultiIndex]
- 기본 인덱스
    * .set_index() <- 컬럼을 인덱스로 설정
    * .reindex() <- 인덱스 재구성(일부 변경, 결측치 발생 가능)
    * .reset_index() <- 위치 인덱스(RangeIndex)로 변경. 기존 인덱스는 컬럼 추가
    * .iloc[위치인덱스] 속성 <- 판다스에서 DataFrame/Series에 데이터가 존재하는 위치 번호를 지정/정수
    * .loc[] 속성 <- DataFrame/Series 생성 시 index, columns 매개변수로 지정한 인덱스들
- 멀티 인덱스
    * .xs() <- 인덱스 레벨에 따른 데이터 선택하는 메서드

[1] 모듈 로딩 및 데이터 준비 <hr>

In [26]:
## 모듈 로딩
import pandas as pd

In [27]:
## 데이터 준비
DATA_FILE = '../Data/학생관리부.xlsx'

## index_col = int, str, sequence int, slicing
dataDF = pd.read_excel(DATA_FILE, skiprows=2, index_col=[1, 2])

In [28]:
## 확인
display(dataDF.head())

이름   석차
학년 반          
1  1  홍길동    8
   3  고길동    1
   7  이길동   12
3  1  박길동    9
2  1  최길동   43

[2] DF에서 인덱스 관련 메서드 다루기 <hr>

In [35]:
## =========================================================
## 멀티인덱스 속성 확인
## =========================================================
mIDX = dataDF.index
print('--------------------------------------')
print(f'type(mIDX): {type(mIDX)}')
print(f'names     : {mIDX.names}')
print(f'levels    : {mIDX.levels}')
print(f'dtypes    : {mIDX.dtypes}')
print(f'nlevels   : {mIDX.nlevels}개')
print('--------------------------------------')
print(f'levshape  : {mIDX.levshape}')
print(mIDX)

--------------------------------------
type(mIDX): <class 'pandas.core.indexes.multi.MultiIndex'>
names     : ['학년', '반']
levels    : [[1, 2, 3], [1, 3, 7]]
dtypes    : 학년    int64
반     int64
dtype: object
nlevels   : 2개
--------------------------------------
levshape  : (3, 3)
MultiIndex([(1, 1),
            (1, 3),
            (1, 7),
            (3, 1),
            (2, 1)],
           names=['학년', '반'])


In [40]:
## ==============================================================
## [2-1] 행 데이터 선택
## ==============================================================
# MultiIndex([(1, 1),
#             (1, 3),
#             (1, 7),
#             (3, 1),
#             (2, 1)],
#            names=['학년', '반'])

## - 전체 행인덱스로 선택
oneSR = dataDF.loc[(1, 7)]
print(f'\ndataDF.loc[(1, 7)]\n{oneSR}\n]')

## - level0번 행인덱스로 선택 => level0 동일한 모든 행
level0 = dataDF.loc[(1)]
print(f'\ndataDF.loc[(1)]\n{level0}\n]')

## - .loc[]/.iloc[] : 멀티인덱스에서 레벨 순서대로 저장했을 때만 가능
## - level1번 행인덱스로 선택 => level1 동일한 모든 행
## => xs()메서드 사용
level1 = dataDF.xs(7, level='반')
print(f'\ndataDF.xs(7)\n{level1}\n]')


dataDF.loc[(1, 7)]
이름     이길동
 석차     12
Name: (1, 7), dtype: object
]

dataDF.loc[(1)]
    이름   석차
반          
1  홍길동    8
3  고길동    1
7  이길동   12
]

dataDF.xs(7)
     이름   석차
학년          
1   이길동   12
]


In [41]:
display(dataDF)

이름   석차
학년 반          
1  1  홍길동    8
   3  고길동    1
   7  이길동   12
3  1  박길동    9
2  1  최길동   43

In [44]:
## [실습] 1반 친구들만 선택하기
first = dataDF.xs(1, level='반')
print(first)

     이름   석차
학년          
1   홍길동    8
3   박길동    9
2   최길동   43


In [47]:
## ==============================================================
## [2-2] 행 추가 => 사전식 정렬 추천/성능 느려지는 것을 막기 위해서
## ==============================================================
## => 모든 컬럼 동일값 추가
dataDF.loc[(2, 2), :] = 0
dataDF.sort_index(inplace=True)     # 권장사항
dataDF

이름    석차
학년 반           
1  1  홍길동   8.0
   3  고길동   1.0
   7  이길동  12.0
2  1  최길동  43.0
   2    0   0.0
3  1  박길동   9.0

In [50]:
## => 모든 컬럼 다른값 추가
dataDF.loc[(2, 3), :] = ['배트맨', 2.3]
dataDF.sort_index(inplace=True)
dataDF

이름    석차
학년 반           
1  1  홍길동   8.0
   3  고길동   1.0
   7  이길동  12.0
2  1  최길동  43.0
   2    0   0.0
   3  배트맨   2.3
3  1  박길동   9.0

In [52]:
## ==============================================================
## [2-2] 열 추가
## ==============================================================
dataDF['번호'] = 0
dataDF.head(2)

이름   석차  번호
학년 반              
1  1  홍길동  8.0   0
   3  고길동  1.0   0

In [54]:
dataDF['학교'] = ['대구', '대구', '부산', '대전', '마산', '대구', '서울']
dataDF

이름    석차  번호  학교
학년 반                   
1  1  홍길동   8.0   0  대구
   3  고길동   1.0   0  대구
   7  이길동  12.0   0  부산
2  1  최길동  43.0   0  대전
   2    0   0.0   0  마산
   3  배트맨   2.3   0  대구
3  1  박길동   9.0   0  서울

In [56]:
## ==============================================================
## [2-3] 행 인덱스 초기화
## ==============================================================
## -> 모든 멀티인덱스 => 컬럼으로 추가
dataDF.reset_index()

,학년,반,이름,석차,번호,학교
0,1,1,홍길동,8.0,0,대구
1,1,3,고길동,1.0,0,대구
2,1,7,이길동,12.0,0,부산
3,2,1,최길동,43.0,0,대전
4,2,2,0,0.0,0,마산
5,2,3,배트맨,2.3,0,대구
6,3,1,박길동,9.0,0,서울


In [57]:
## -> 특정 레벨 행 인덱스만 => 컬럼으로
dataDF.reset_index(level=['반'])

,반,이름,석차,번호,학교
학년,,,,,
1,1,홍길동,8.0,0,대구
1,3,고길동,1.0,0,대구
1,7,이길동,12.0,0,부산
2,1,최길동,43.0,0,대전
2,2,0,0.0,0,마산
2,3,배트맨,2.3,0,대구
3,1,박길동,9.0,0,서울
